In [9]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import time

from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.metrics import mean_squared_error

from math import sqrt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [10]:
#Importing dataset
df = pd.read_csv("dataset/train.csv")
df_val = pd.read_csv("dataset/train_val1.csv")

In [11]:
print(df.shape)
print(df_val.shape)

(334212, 44)
(62835, 44)


We normalize the data so that all values are between 0 and 1. So we first remove 'Date' which is the only non numerical attribute. Then we apply range normalization and at the end we add 'Date'. 

In [12]:
df= (df - df.min()) / (df.max() - df.min())
df.head(5)

,Unnamed: 0,StoreID,IsHoliday,HasPromotions,StoreType,NearestCompetitor,NumberOfSales,Region_AreaKM2,Region_GDP,Region_PopulationK,...,Snow,Fog,Hail,Thunderstorm,IsSaturday,IsSunday,WasOpenYesterday,IsOpenTomorrow,MeanMonthSales,MeanStoreSales
0,0.000000,0.0,0.0,0.0,0.666667,0.003281,0.213055,0.291715,0.515529,0.266576,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.311753,0.430061
1,0.000002,0.0,0.0,0.0,0.666667,0.003281,0.304456,0.291715,0.515529,0.266576,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.311753,0.430061
2,0.000004,0.0,0.0,0.0,0.666667,0.003281,0.311550,0.291715,0.515529,0.266576,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.311753,0.430061
3,0.000006,0.0,0.0,0.0,0.666667,0.003281,0.268533,0.291715,0.515529,0.266576,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.311753,0.430061
4,0.000010,0.0,0.0,1.0,0.666667,0.003281,0.379490,0.291715,0.515529,0.266576,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.311753,0.430061


We try to find the best number of features

In [20]:
y = df['NumberOfSales']
y_val = df_val['NumberOfSales']
X = df.drop(['NumberOfSales'], axis=1)
X = df.drop(['NumberOfSales'], axis=1)

In [14]:
#X_t = df_train_clear.loc[df_train_clear['StoreID'] == 0.0]
#y_t = X_t['NumberOfSales']
#y_val_t = df_val1_clear.loc[df_val1_clear['StoreID'] == 0.0]['NumberOfSales']
#X_t = X_t.drop(['StoreID','NumberOfSales','NumberOfCustomers','YesterdaySales','Previous3DaysSales','PreviousWeekSales','PreviousMonthSales'], axis=1)

In [15]:
#y_val_t.shape

In [16]:
dict_df = {'loss_function' : [], 'alpha' : [], 'epsilon' : [], 'learning_rate' : [], 'var_threshold' : [],'num_feat' : [], 'mse' : []}

In [21]:
loss_fun = ['squared_loss', 'epsilon_insensitive']
learning_r = ['constant', 'optimal', 'invascaling']

start = time.time()

for l_f in loss_fun:
    for l_r in learning_r:
        for a in np.arange(.0001, .01 , .0001):
            for e in np.arange(.1, .5, .1):
                sgd = SGDRegressor(loss=l_f, alpha=a, epsilon=e, learning_rate=l_r, shuffle=False, random_state=1234567890)
                dict_df['loss_function'].append(l_f)
                dict_df['alpha'].append(a)
                dict_df['epsilon'].append(e)
                dict_df['learning_rate'].append(l_r)
                for v in np.arange(.001, .03, .0005):
                    sfm = SelectFromModel(sgd, threshold=v)
                    sfm.fit(X, y)
                    n_features = sfm.transform(X).shape[1]
                    dict_df['var_threshold'].append(v)
                    dict_df['num_feat'].append(n_features)
                    if n_features == 0:
                        dict_df['mse'].append(99999999999999999)
                    else:
                        mask = sfm.get_support()
                        X_mask = X.loc[:, mask]
                        X_val1_mask = X_val1.loc[:, mask]
                        lasso.fit(X_mask, y)
                        y_pred=lasso.predict(X_val1_mask)
                        dict_df['mse'].append(sqrt(mean_squared_error(y_val1,y_pred)))
            
end = time.time() - start
end

NameError: name 'X_val1' is not defined

In [ ]:
pd.DataFrame(dict_df).to_csv("res_sgd_1.csv")